In [ ]:
import requests
from bs4 import BeautifulSoup
import os
from tqdm.notebook import tqdm
import pandas as pd

path = os.getcwd()
# Specify the category you want to search in
# category = "herren/schuhe/sneaker"  # replace with the category you want

# Define the URL of the site
url = f"https://www.vinted.de/"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

# Send HTTP request to site and save the response from server in a response object called r
r = requests.get(url, headers=headers)

# Create a BeautifulSoup object and specify the parser
soup = BeautifulSoup(r.text, 'html.parser')

# Extract the desired info (the offers in this case)
# The selector depends on the structure of the webpage, inspect the webpage to find the right selector
offers = soup.select('div.offer')


# Get response code
print(r)

# Get HTML text
page = requests.get(url)
index_page = page.text
# print(page.text)
with open ("test.txt",'w') as f:
    f.write((str(page.text)))

## Obtain different category urls for more detailed scraping


In [ ]:
def find_all(string, substring):
    start = 0
    while True:
        start = string.find(substring, start)
        if start == -1: return
        end = start + 100
        yield string[start:end]
        start += len(substring) # use start += 1 to find overlapping matches


# def 

substring_damen, substring_herren = "/damen", "/herren"


cat_urls_uncut_damen = set(find_all(page.text, substring_damen))
cat_urls_uncut_herren = set(find_all(page.text, substring_herren))



cat_urls_cut_damen, cat_urls_cut_herren, cat_names_damen, cat_names_herren = [],[],[],[]


for i in cat_urls_uncut_damen:
    i = i.split(sep='\"')
    cat_urls_cut_damen.append("https://www.vinted.de{}".format(i[0]))
    cat_names_damen.append(i[0])
    # print(i)

for i in cat_urls_uncut_herren:
    i = i.split(sep='\"')
    i = "https://www.vinted.de{}".format(i[0])

print("We found {} categories for women on vinted that we can scrape".format(len(cat_urls_uncut_damen)))
print("We found {} categories for men vinted that we can scrape".format(len(cat_urls_uncut_herren)))
for name in cat_names_damen:
    print(name)
    



In [ ]:
print(cat_urls_cut_damen)
print(cat_names_damen)
print(len(cat_urls_cut_damen))

## Get HTMLs of all subcategories

Since the HTML is not consistent with the use of variable like titles, id etc. we need to obtain the display names of the categories we want to get data from. For that reason we format the names given in the headers of the category pages. this may seem not as straight forward but is the easiest way to actually get the IDs and categories we want to obtain. Call it a workaround :)


In [ ]:
sub_path = path + '/categories'
if not os.path.exists(sub_path):
    os.makedirs(sub_path)
print(sub_path)



display_titles = []

for url, name in tqdm(zip(cat_urls_cut_damen,cat_names_damen)):
    # Send HTTP request to site and save the response from server in a response object called r
    r = requests.get(url)
    
    # Get the index of display title 
    index_title = r.text.find("<title>")
    title_uncut = r.text[index_title:index_title+100]
    splt = title_uncut.split(sep=" |")
    display_titles.append(splt[0])

    # Create a BeautifulSoup object and specify the parser      
    soup = BeautifulSoup(r.text, 'html.parser')
    filename = "%s.txt" % name[1:]
    filename = filename.replace("/","_")
    # print(filename)
    # print(type(filename))


In [ ]:
titles_cut = []
for title in display_titles:
    title = title[7:]
    title = title.replace("&amp;","\\u0026")
    titles_cut.append(title)
print(len(titles_cut))


In [ ]:
with open ("list_names.txt", 'w') as f:
    for i in titles_cut:
        f.writelines(i + '\n')

In [ ]:
doubles = []
for i in titles_cut:
    c = titles_cut.count(i)

        # print("Title: {} occured {} times.".format(i,c))
    doubles.append((i,c))

set_d = set(doubles)
for i in set_d:
    print(i)
names = list(set_d)

## Retrieve the overview Data

In [ ]:
# names = set(titles_cut)
properties = ['id','title','code','item_count','url']


def get_single_property(str:str,p:str):
    i = str.find(p)
    sub = str[i:]
    sub = sub.split(",")
    sub = sub[0]
    prop_len = len(p)
    name = sub[:prop_len]
    sub = sub[prop_len+1:]
    return name, sub



def get_dataframe(names):

    for i in names:
        start = 0
        for j in range(i[1]):
            
            list_name = []
            dict_props = {}
            ind = index_page.find(i[0], start)
            
            # Get rough length to be cut 
            sub = index_page[ind-20:ind+1000]
            
            # Assign the new start for str.find() as to not retrieve the same index 2x
            start = ind + len(sub)


            sub = sub.replace("{","")
            sub = sub.replace("\"","")

            # Obtain all single desired properties of substring
            for p in properties:
                name, value = get_single_property(sub, p)
                dict_props[p] = value


            # convert the dictionary to a pandas DataFrame
            df = pd.DataFrame(dict_props, index=[0])
            yield df


dfs = list(get_dataframe(names))
combined = pd.concat(dfs)

# Check that there is no duplicates in list and have a look into the df
print( True in combined['id'].duplicated())
print(combined.head())


## Category specific retrieving of data

Syntax of the urls to retrieve from  : https://www.vinted.de/catalog?catalog[]=1070&time=1690115802&page=2

In [ ]:
def get_html_from_url(url):


    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}

    # Send HTTP request to site and save the response from server in a response object called r
    r = requests.get(catalog_url, headers=headers)

    # Create a BeautifulSoup object and specify the parser
    soup = BeautifulSoup(r.text, 'html.parser')

    # Extract the desired info (the offers in this case)
    # The selector depends on the structure of the webpage, inspect the webpage to find the right selector
    offers = soup.select('div.offer')


    # Get HTML text
    page = requests.get(catalog_url)
    return page.text

In [ ]:
def get_items_ids(page):
    length = 1500
    s = "{\"catalogItems\":{\"ids\":"
    index = page.find(s)
    sub = page[index:index+length]

    # Remove everything around the ids
    splts = sub.split("[")
    sub = splts[1]
    splts = sub.split("]")
    sub = splts[0]
    
    splt = sub.split(",")
    for i in splt:
        yield i

In [95]:
def get_dataframe_category(html,subs):

    for i in subs:
        dict_props = {}
        ind = html.find(i)
        
        # Get rough length to be cut 
        sub = html[ind:ind+5000]
        splt = sub.split("\",\"content_source\"")
        sub = splt[0]    


        sub = sub.replace("{","")
        sub = sub.replace("\"","")

        # Obtain all single desired properties of substring
        for p in properties_announce:
            name, value = get_single_property(sub, p)
            dict_props[p] = value


        # convert the dictionary to a pandas DataFrame
        df = pd.DataFrame(dict_props, index=[0])
        # print(df.head())

        yield df

In [96]:
from loguru import logger
import time
catalog_url = "https://www.vinted.de/catalog?catalog[]=1070&page=3"



def get_announces_by_category(category_id, nr_pages):

    properties_announce = ['id','title','price','brand_title','size_title','user:id','login','profile_url'] 

    for page in tqdm(range(nr_pages_to_scrape)):
        try:
                
            url = "https://www.vinted.de/catalog?catalog[]={}&page{}".format(category_id,page+1)
            # print(url)
            html_string = get_html_from_url(url)
            # print(html_string)

            l = list(get_items_ids(html_string))
            unique_id_references = ["\"id\":"+i for i in l]
            # print(unique_id_references)

            announces = list(get_dataframe_category(html=html_string, subs=unique_id_references))

            announces_df = pd.concat(announces)
            print(announces_df.head())
            time.sleep(2)
            yield announces_df
            
        except Exception as e:
            logger.exception(e)


category_id = 1070
nr_pages_to_scrape = 10
list_category_announces = list(get_announces_by_category(category_id,nr_pages_to_scrape))
df_category_announces = pd.concat(list_category_announces)




  0%|          | 0/10 [00:00<?, ?it/s]

           id                       title price brand_title   size_title  \
0  3036760663  Mango Suits black trousers   5.0       Mango  M / 38 / 10   

    user:id      login                                      profile_url  
0  94302955  benoancsi  https://www.vinted.de/member/94302955-benoancsi  
           id                       title price brand_title     size_title  \
0  3201870021  Cropped Geometric Trousers  10.0     Intrend  XXXS / 30 / 2   

    user:id      login                                      profile_url  
0  44722615  mossmabon  https://www.vinted.de/member/44722615-mossmabon  
           id           title price brand_title   size_title   user:id  \
0  3173471210  Balcone pants    2.0     Bershka  M / 38 / 10  53880348   

              login                                        profile_url  
0  lorenzocolella97  https://www.vinted.de/member/53880348-lorenzoc...  
           id                                      title price brand_title  \
0  3014582640  Velvet

KeyboardInterrupt: 

In [ ]:
print(df_category_announces.shape[0])
print(df_category_announces.head())
